In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from Bio.KEGG.KGML.KGML_parser import read
import pandas as pd
import os
from collections import defaultdict
from tqdm import tqdm
import requests
import networkx as nx
import numpy as np

In [2]:
# url = 'http://rest.kegg.jp/link/hsa/pathway'
# html = urlopen(url).read().decode("utf-8")

In [3]:
# soup = BeautifulSoup(html, 'html.parser')

In [4]:
# finding KEGG ID's of every gene (?) here
kegg_dict = {
    'Zap70' : 7535,
    'SHP2' : 5781,
    'Stat1' : 6772,
    'Akt' : 207,
    'Stat5a' : 6776,
    'Stat5b' : 6777,
    'p38' : 1432,
    'NFkB1' : 4790,
    'NFKB2' : 4791,
    'NFkBIA' : 4792,
    'S6' : 6194,
    'Lat' : 27040,
    'Erk' : 5594,
    'Plcg2' : 5336,
    'Btk' : 695,
    'Slp76' : 3937,
    'Stat3' : 6774
}

In [6]:
# pathways = []
# for line in html.split('\n'):
#     for kegg_id in kegg_dict.values():
#         if ('hsa:' + str(kegg_id)) in line:
#             pathway = line.split('\t')[0]
#             pathways.append(pathway)
# pathways = list(set(pathways))

In [7]:
# len(pathways)

In [8]:
# pathways[:5]

In [9]:
# # files persisted
# for pathway in tqdm(pathways):
#     url = 'http://rest.kegg.jp/get/' + pathway.split(':')[1] + '/kgml'
#     response = requests.get(url)
#     with open('pathways/' + pathway.split(':')[1] + '.xml', 'wb') as file:
#         file.write(response.content)

In [10]:
test_pathway = read(open('pathways/hsa04270.xml', 'r'))

In [11]:
test_pathway.relations[0].entry1.name

'hsa:5592'

In [12]:
gene_hsa = ['hsa:' + str(val) for val in kegg_dict.values()]
gene_hsa_dict = {('hsa:' + str(v)) : k for k,v in kegg_dict.items()}

In [13]:
link_list = []
for kgml in tqdm(os.listdir('pathways/')):
    # check for xml
    if kgml[-4:] != '.xml':
        continue
    path_name = kgml[:-4]
    
    # read file
    path = read(open('pathways/' + kgml, 'r'))
    
    # iterate through edges
    for rel in path.relations:
        link_list.append({
            'pathway' : path_name,
            'entry1' : rel.entry1.name,
            'entry2' : rel.entry2.name,
            'type' : rel.type,
            'subtype' : rel.subtypes
        })

100%|███████████████████████████████████████████████████████████████████████████████| 167/167 [00:01<00:00, 139.71it/s]


In [14]:
links = pd.DataFrame(link_list)
links = links[(links['entry1'] != 'undefined') & (links['entry2'] != 'undefined')]
links

,pathway,entry1,entry2,type,subtype
0,hsa00010,hsa:218 hsa:221 hsa:222,hsa:10327,ECrel,"[(compound, 105)]"
1,hsa00010,hsa:83440,hsa:2821,ECrel,"[(compound, 93)]"
2,hsa00010,hsa:5313 hsa:5315,hsa:2023 hsa:2026 hsa:2027 hsa:387712,ECrel,"[(compound, 96)]"
3,hsa00010,hsa:130589,hsa:83440,ECrel,"[(compound, 90)]"
4,hsa00010,hsa:2203 hsa:8789,hsa:2821,ECrel,"[(compound, 95)]"
...,...,...,...,...,...
12005,hsa05418,hsa:10365,hsa:8878,GErel,"[(expression, -->)]"
12006,hsa05418,hsa:4780,hsa:1728,GErel,"[(expression, -->)]"
12007,hsa05418,hsa:2353 hsa:3725,hsa:7124,GErel,"[(expression, -->)]"
12008,hsa05418,hsa:2353 hsa:3725,hsa:3383,GErel,"[(expression, -->)]"


In [15]:
# create a graph for each pathway from links
all_paths = {}
for kegg_path in tqdm(np.unique(links['pathway'])):
    G = nx.DiGraph()
    for i in list(links[links['pathway'] == kegg_path].index):
        row = links.loc[i]
        G.add_edge(row['entry1'], row['entry2'])
    all_paths[kegg_path] = (nx.algorithms.shortest_path(G))

100%|███████████████████████████████████████████████████████████████████████████████| 160/160 [00:01<00:00, 119.04it/s]


In [17]:
counter = 0
for pathway in tqdm(all_paths):
    pathway_paths = all_paths[pathway]
    keys = list(pathway_paths.keys())
    for key in keys:
        # check if source node is relevant protein
        relevant = False
        for gene in gene_hsa:
            if gene in key:
                relevant = True
                # now check if target relevant
                for key2 in list(pathway_paths[key].keys()):
                    relevant2 = False
                    for gene in gene_hsa:
                        if gene in key2:
                            relevant2 = True
                    if not relevant2:
                        # if target not relevant, remove it from dict
                        pathway_paths[key].pop(key2)
        if not relevant:
            # if source not relevant, remove it from dict
            pathway_paths.pop(key)

100%|██████████████████████████████████████████████████████████████████████████████| 160/160 [00:00<00:00, 4196.51it/s]


In [18]:
def find_node(node_name):
    '''
    Given a string like 'hsa:2023 hsa:2026 hsa:2027 hsa:387712', return which proteins match
    '''
    proteins = []
    for protein in gene_hsa_dict.keys():
        if protein in node_name:
            proteins.append(gene_hsa_dict[protein])
    return proteins

In [19]:
relevant_links = []
for pw in all_paths.keys():
    pathway = all_paths[pw]
    for node1 in pathway.keys():
        for node2 in pathway[node1].keys():
            node1_proteins = find_node(node1)
            node2_proteins = find_node(node2)
            for protein in node1_proteins:
                for protein2 in node2_proteins:
                    relevant_links.append({
                        'node1' : protein,
                        'node2' : protein2,
                        'pathway' : pw,
                        'path': pathway[node1][node2]
                    })
relevant_links = pd.DataFrame(relevant_links)
relevant_links = relevant_links[relevant_links['node1'] != relevant_links['node2']].reset_index(drop = True)
relevant_links.to_csv('kegg_links.csv', index = False)

In [20]:
relevant_links

,node1,node2,pathway,path
0,Plcg2,Erk,hsa01521,"[hsa:5335 hsa:5336, cpd:C00165, hsa:5578 hsa:5..."
1,Akt,S6,hsa01521,"[hsa:10000 hsa:207 hsa:208, hsa:2475, hsa:6198..."
2,Akt,Erk,hsa01522,"[hsa:10000 hsa:207 hsa:208, hsa:2475, hsa:6198..."
3,p38,Akt,hsa01522,"[hsa:1432 hsa:5600 hsa:5603 hsa:6300, hsa:1049..."
4,p38,Erk,hsa01522,"[hsa:1432 hsa:5600 hsa:5603 hsa:6300, hsa:1049..."
...,...,...,...,...
533,Stat1,Stat3,hsa05321,"[hsa:6772, hsa:30009, hsa:3595, hsa:8807 hsa:8..."
534,NFkBIA,NFkB1,hsa05417,"[hsa:4792, hsa:4790 hsa:5970]"
535,p38,NFkB1,hsa05417,"[hsa:1432 hsa:5600 hsa:5603 hsa:6300, hsa:4790..."
536,Erk,NFkB1,hsa05417,"[hsa:5594 hsa:5595, hsa:4790 hsa:5970]"
